In [ ]:
#Импорт библиотек
import os
import mat_properties as prop
import numpy as n
import pandas as pd
import cotel
import nasos
import time
from scipy.optimize import root

#Время начала расчёта


#Основные констант3
calctolerance=10**-3
calcmethod = "hybr"
KPDnasos = 0.8074
KPD=0.99

#таблица номинального режима
gas_streams0   = pd.read_excel('streams0.xlsx', sheet_name='gas',  index_col=0)
water_streams0 = pd.read_excel('streams0.xlsx', sheet_name='water',index_col=0)
#таблица задаваемых значений (заменится на интерфейс)
gas_streamsZAD    = pd.read_excel('streams.xlsx',  sheet_name='gas',  index_col=0)
water_streamsZAD  = pd.read_excel('streams.xlsx',  sheet_name='water',index_col=0)
#рабочая таблица (=номинал в 1 итерации)
gas_streams = gas_streams0.copy()
water_streams = water_streams0.copy()

#чтение с вводного экселя входных данных
Tgas = gas_streamsZAD.loc['GTU-PEVD','T']
Pgas = gas_streamsZAD.loc['GTU-PEVD','P']
Ggas = gas_streamsZAD.loc['GTU-PEVD','G']
Propgas = gas_streamsZAD.loc['GTU-PEVD','N2':'Ar']
Pvd = water_streamsZAD.loc['IVD-PEVD','P']
Pnd = water_streamsZAD.loc['IND-PPND','P']
Twat0 = water_streamsZAD.loc['X-GPK','T']

#запись состава газов в раб табл
gas_streams['N2'] = Propgas[0]
gas_streams['O2'] = Propgas[1]
gas_streams['CO2'] = Propgas[2]
gas_streams['H2O'] = Propgas[3]
gas_streams['Ar'] = Propgas[4]

#Состав газов в номинальном режиме и в частичной нагрузке
gasmix = "Nitrogen*Oxygen*CO2*Water*Argon"
fractiongas0 = (gas_streams0.at['GTU-PEVD','N2'],gas_streams0.at['GTU-PEVD','O2'],gas_streams0.at['GTU-PEVD','CO2'],gas_streams0.at['GTU-PEVD','H2O'],gas_streams0.at['GTU-PEVD','Ar'])
fractiongas  = (gas_streams.at['GTU-PEVD','N2'],gas_streams.at['GTU-PEVD','O2'],gas_streams.at['GTU-PEVD','CO2'],gas_streams.at['GTU-PEVD','H2O'],gas_streams.at['GTU-PEVD','Ar'])

#Считывание рефпропа
RP = prop.init_REFPROP(r'C:\Program Files (x86)\REFPROP')
os.environ['RPPREFIX'] = r'C:/Program Files (x86)/REFPROP'
gas1 = prop.Materials_prop(gasmix,fractiongas, prop.REFPROP_h_s,prop.REFPROP_p_t,prop.REFPROP_p_h,prop.REFPROP_p_s,prop.REFPROP_p_q,prop.REFPROP_t_q,RP=RP)
gas0 = prop.Materials_prop(gasmix,fractiongas0, prop.REFPROP_h_s,prop.REFPROP_p_t,prop.REFPROP_p_h,prop.REFPROP_p_s,prop.REFPROP_p_q,prop.REFPROP_t_q,RP=RP)
water=prop.Materials_prop('water',[1.0,0,0,0,0], prop.REFPROP_h_s,prop.REFPROP_p_t,prop.REFPROP_p_h,prop.REFPROP_p_s,prop.REFPROP_p_q,prop.REFPROP_t_q,RP=RP)

#Задание в раб табл параметров газа (1 строка)
gas_streams.loc['GTU-PEVD','T'] = Tgas
gas_streams['G'] = Ggas
gas_streams['P'] = Pgas
gas_streams.loc['GTU-PEVD','H'] = gas1.p_t(Pgas,Tgas)['h']

#Задание в раб табл параметров по воде
water_streams.loc['IVD-PEVD':'PEN-EVD','P'] = Pvd
water_streams.loc['IND-PPND':'X-GPK','P'] = Pnd
water_streams.loc['BND-PEN','P'] = Pnd
water_streams.loc['X-GPK','T'] = Twat0

#Расчёт параметров на насыщении в испарительных поверхностях (в 1 итерации приближение: недогрев до н.п. на входе 3°С)
water_streams.loc['IVD-PEVD','T'] = water.p_q(Pvd,1)['T']
water_streams.loc['IVD-PEVD','H'] = water.p_q(Pvd,1)['h']
water_streams.loc['IND-PPND','T'] = water.p_q(Pnd,1)['T']
water_streams.loc['IND-PPND','H'] = water.p_q(Pnd,1)['h']
water_streams.loc['EVD-IVD','T'] = water.p_q(Pvd,1)['T']-3
water_streams.loc['GPK-IND','T'] = water.p_q(Pnd,1)['T']-3
water_streams.loc['EVD-IVD','H'] = water.p_t(Pvd,water_streams.loc['EVD-IVD','T'])['h']
water_streams.loc['GPK-IND','H'] = water.p_t(Pnd,water_streams.loc['GPK-IND','T'])['h']

#Задача моделей

PEVD_obj = cotel.heatex('GTU-PEVD','PEVD-IVD','IVD-PEVD','PEVD-DROS',
                    KPD,calctolerance,gas0,gas1,water,calcmethod,gas_streams0,water_streams0,gas_streams,water_streams)

IVD_obj  = cotel.evaporVD ('PEVD-IVD','IVD-EVD','EVD-IVD','IVD-PEVD',          
                           KPD,calctolerance,gas1,gas0,water,calcmethod,gas_streams0,water_streams0,gas_streams,water_streams)

EVD_obj = cotel.heatex('IVD-EVD','EVD-PPND','PEN-EVD','EVD-IVD',               
                   KPD,calctolerance,gas0,gas1,water,calcmethod,gas_streams0,water_streams0,gas_streams,water_streams)


PEN_obj =nasos.nasos('BND-PEN','PEN-EVD',water,KPDnasos,water_streams)


PPND_obj = cotel.heatexPEND('EVD-PPND','PPND-IND','IND-PPND','PPND-TURB',    
                            KPD,calctolerance,gas0,gas1,water,calcmethod,gas_streams0,water_streams0,gas_streams,water_streams)

IND_obj  = cotel.evaporND('PPND-IND','IND-GPK','GPK-IND','IND-PPND',      
                             KPD,calctolerance,gas1,gas0,water,calcmethod,gas_streams0,water_streams0,gas_streams,water_streams)

GPK_obj = cotel.heatex('IND-GPK','GPK-out','X-GPK','GPK-IND',
                       KPD,calctolerance,gas1,gas0,water,calcmethod,gas_streams0,water_streams0,gas_streams,water_streams)
start_time = time.time()
#Расчёт
it=60
for i in range(it):
    ######################################################################
    #Связвка высокого давления
    for j in range(it):
        
        #Расчёт ПЕВД
        PEVD = PEVD_obj.calc()
        gas_streams.loc['PEVD-IVD','T':'G'] = [PEVD[0],PEVD[1],PEVD[2],PEVD[3]]
        water_streams.loc['PEVD-DROS','T':'G'] = [PEVD[4],PEVD[5],PEVD[6],PEVD[7]]
        
        #Расчёт ИВД
        IVD  =       IVD_obj.calc()
        gas_streams.loc['IVD-EVD','T':'G'] = [IVD[0],IVD[1],IVD[2],IVD[3]]
        water_streams.loc['IVD-PEVD','T':'G'] = [IVD[4],IVD[5],IVD[6],IVD[7]]
        
        #Переопределение расхода в ВД
        water_streams.loc['PEVD-DROS':'PEN-EVD','G'] = IVD[7]
        water_streams.loc['BND-PEN','G'] = IVD[7]

        #Расчёт ЭВД
        EVD = EVD_obj.calc()
        gas_streams.loc['EVD-PPND','T':'G'] = [EVD[0],EVD[1],EVD[2],EVD[3]]
        water_streams.loc['EVD-IVD','T':'G'] = [EVD[4],EVD[5],EVD[6],EVD[7]]
        
        #Баланс ПЕВ+ИВД+ЭВД
        Qgas1 = KPD*gas_streams.at['GTU-PEVD','G']*(gas_streams.at['GTU-PEVD','H']-gas_streams.at['IVD-EVD','H'])
        Qwat1 = water_streams.at['PEVD-DROS','G']*(water_streams.at['PEVD-DROS','H']-water_streams.at['EVD-IVD','H'])
        print('dQ/Q ПЕВД+ИВД+ЭВД',((Qgas1-Qwat1)/Qgas1*100))
        if  abs((Qgas1-Qwat1)/Qgas1*100) < calctolerance:
            break;
    print("ВД: --- %s сек. ---" % round((time.time() - start_time),1))
    ######################################################################
    
    #Для сходимости
    if i==0:
        gas_streams.loc['PPND-IND','T'] = gas_streams.loc['EVD-PPND','T'] -1
        gas_streams.loc['PPND-IND','H'] = gas1.p_t(gas_streams.loc['PPND-IND','P'],gas_streams.loc['PPND-IND','T'])['h']
    
    ######################################################################
    #Связка низкого давления
    for j in range(it): 
        #Расчёт ППНД
        PPND = PPND_obj.calc()
        gas_streams.loc['PPND-IND','T':'G']    = [PPND[0],PPND[1],PPND[2],PPND[3]]
        water_streams.loc['PPND-TURB','T':'G'] = [PPND[4],PPND[5],PPND[6],PPND[7]]
        
        #Расчёт ИНД
        IND=IND_obj.calc()
        gas_streams.loc['IND-GPK','T':'G']    = [IND[0],IND[1],IND[2],IND[3]]
        water_streams.loc['IND-PPND','T':'G'] = [IND[4],IND[5],IND[6],IND[7]]
        
        #Переопределение расхода в НД
        water_streams.loc['PPND-TURB':'IND-PPND','G'] = IND[7]
        
        #ПЭН
        water_streams.loc['BND-PEN','T':'G'] = [IND[9],IND[10],IND[11],IND[12]]
        PEN=PEN_obj.calc()
        water_streams.loc['PEN-EVD','T':'G'] = [PEN[0],PEN[1],PEN[2],PEN[3]]
        
        #Баланс ППНД+ИНД
        Qgas = KPD*gas_streams.at['EVD-PPND','G']*(gas_streams.at['EVD-PPND','H']-gas_streams.at['IND-GPK','H'])
        Qwat = water_streams.at['IND-PPND','G']*(water_streams.at['IND-PPND','H']-water_streams.at['GPK-IND','H'])+\
        water_streams.at['PPND-TURB','G']*(water_streams.at['PPND-TURB','H']-\
        water_streams.at['IND-PPND','H'])+water_streams.at['BND-PEN','G']*(water_streams.at['BND-PEN','H']-water_streams.at['GPK-IND','H'])
        
        #Расчёт расхода в ГПК
        water_streams.loc['X-GPK','G'] = water_streams.at['PPND-TURB','G']+water_streams.at['PEVD-DROS','G']
        water_streams.loc['GPK-IND','G'] = water_streams.at['X-GPK','G']
        
        #Расчёт ГПК
        GPK = GPK_obj.calc()
        gas_streams.loc['GPK-out','T':'G']   = [GPK[0],GPK[1],GPK[2],GPK[3]]
        water_streams.loc['GPK-IND','T':'G'] = [GPK[4],GPK[5],GPK[6],GPK[7]]
        
        #Баланс ППНД+ИНД+ГПК
        Qgas1 = KPD*gas_streams.at['EVD-PPND','G']*(gas_streams.at['EVD-PPND','H']-gas_streams.at['GPK-out','H'])
        Qwat1 = water_streams.at['GPK-IND','G']*(water_streams.at['GPK-IND','H']-water_streams.at['X-GPK','H'])+\
        water_streams.at['IND-PPND','G']*(water_streams.at['IND-PPND','H']-water_streams.at['GPK-IND','H'])+\
        water_streams.at['PPND-TURB','G']*(water_streams.at['PPND-TURB','H']-water_streams.at['IND-PPND','H'])+\
        water_streams.at['BND-PEN','G']*(water_streams.at['BND-PEN','H']-water_streams.at['GPK-IND','H'])
        print('dQ/Q ППНД+ИНД+ГПК',(Qgas1-Qwat1)/Qgas1*100)
        if  abs((Qgas1-Qwat1)/Qgas1*100) < calctolerance:
            break;
    print("НД+ --- %s сек. ---" % round((time.time() - start_time),1))
    ######################################################################

    #Баланс общий
    Qgasall = KPD*gas_streams.at['GTU-PEVD','G']*(gas_streams.at['GTU-PEVD','H']-gas_streams.at['GPK-out','H'])
    Qwatall = water_streams.at['PPND-TURB','G']*(water_streams.at['PPND-TURB','H']-water_streams.at['X-GPK','H'])+water_streams.at['PEVD-DROS','G']*(water_streams.at['PEVD-DROS','H']-water_streams.at['X-GPK','H'])-water_streams.at['BND-PEN','G']*(water_streams.at['PEN-EVD','H']-water_streams.at['BND-PEN','H'])
    print('dQ/Qsumm',(Qgasall-Qwatall)/Qgasall*100)
    if  abs((Qgasall-Qwatall)/Qgasall) < calctolerance:
        break;

print('Степень сухости пара в ЭВД: ',water.p_h(water_streams.at['EVD-IVD','P'],water_streams.at['EVD-IVD','H'])['Q'])
print('Степень сухости пара в ГПК: ',water.p_h(water_streams.at['GPK-IND','P'],water_streams.at['GPK-IND','H'])['Q'])
print("fin:--- %s сек. ---" % round((time.time() - start_time),1))

dQ/Q ПЕВД+ИВД+ЭВД -0.3247281597379967
dQ/Q ПЕВД+ИВД+ЭВД -0.5413757137907539
dQ/Q ПЕВД+ИВД+ЭВД -0.3256092132840381
dQ/Q ПЕВД+ИВД+ЭВД -0.28001230470465144
dQ/Q ПЕВД+ИВД+ЭВД -0.2073718996694809
dQ/Q ПЕВД+ИВД+ЭВД -0.1630481807413376
dQ/Q ПЕВД+ИВД+ЭВД -0.125218518443312
dQ/Q ПЕВД+ИВД+ЭВД -0.09711659271906996
dQ/Q ПЕВД+ИВД+ЭВД -0.07505242987798785
dQ/Q ПЕВД+ИВД+ЭВД -0.05809874532715335
dQ/Q ПЕВД+ИВД+ЭВД -0.04495347849041821
dQ/Q ПЕВД+ИВД+ЭВД -0.03479410284276006
dQ/Q ПЕВД+ИВД+ЭВД -0.026930166402711612
dQ/Q ПЕВД+ИВД+ЭВД -0.020845546656336353
dQ/Q ПЕВД+ИВД+ЭВД -0.01613615036214462
dQ/Q ПЕВД+ИВД+ЭВД -0.012491184018944586
dQ/Q ПЕВД+ИВД+ЭВД -0.009669798245714333
dQ/Q ПЕВД+ИВД+ЭВД -0.0074858342046656276
dQ/Q ПЕВД+ИВД+ЭВД -0.005795213808614196
dQ/Q ПЕВД+ИВД+ЭВД -0.004486461024062726
dQ/Q ПЕВД+ИВД+ЭВД -0.0034732997164453254
dQ/Q ПЕВД+ИВД+ЭВД -0.0026889559585535554
dQ/Q ПЕВД+ИВД+ЭВД -0.00208174471251884
dQ/Q ПЕВД+ИВД+ЭВД -0.001611658706966052
dQ/Q ПЕВД+ИВД+ЭВД -0.001247728540333344
dQ/Q ПЕВД+ИВД+ЭВД 

In [ ]:
#Вывод газовых потоков
gas_streams

In [ ]:
#Вывод водяных потоков
water_streams

In [ ]:
#Невязка по всем параметрам
Qgas = KPD*gas_streams.at['GTU-PEVD','G']*(gas_streams.at['GTU-PEVD','H']-gas_streams.at['PEVD-IVD','H'])
Qwat = water_streams.at['PEVD-DROS','G']*(water_streams.at['PEVD-DROS','H']-water_streams.at['IVD-PEVD','H'])
print('dQ/Q ПЕВД+ИВД',((Qgas-Qwat)/Qgas*100))

Qgas1 = KPD*gas_streams.at['GTU-PEVD','G']*(gas_streams.at['GTU-PEVD','H']-gas_streams.at['IVD-EVD','H'])
Qwat1 = water_streams.at['PEVD-DROS','G']*(water_streams.at['PEVD-DROS','H']-water_streams.at['EVD-IVD','H'])
print('dQ/Q ПЕВД+ИВД+ЭВД',((Qgas1-Qwat1)/Qgas1*100))

Qgas = KPD*gas_streams.at['EVD-PPND','G']*(gas_streams.at['EVD-PPND','H']-gas_streams.at['IND-GPK','H'])
Qwat = water_streams.at['IND-PPND','G']*(water_streams.at['IND-PPND','H']-water_streams.at['GPK-IND','H'])+water_streams.at['PPND-TURB','G']*(water_streams.at['PPND-TURB','H']-water_streams.at['IND-PPND','H'])+water_streams.at['BND-PEN','G']*(water_streams.at['BND-PEN','H']-water_streams.at['GPK-IND','H'])
print('dQ/Q ППНД+ИНД',((Qgas-Qwat)/Qgas*100))

Qgas1 = KPD*gas_streams.at['EVD-PPND','G']*(gas_streams.at['EVD-PPND','H']-gas_streams.at['GPK-out','H'])
Qwat1 = water_streams.at['GPK-IND','G']*(water_streams.at['GPK-IND','H']-water_streams.at['X-GPK','H'])+water_streams.at['IND-PPND','G']*(water_streams.at['IND-PPND','H']-water_streams.at['GPK-IND','H'])+water_streams.at['PPND-TURB','G']*(water_streams.at['PPND-TURB','H']-water_streams.at['IND-PPND','H'])+water_streams.at['BND-PEN','G']*(water_streams.at['BND-PEN','H']-water_streams.at['GPK-IND','H'])
print('dQ/Q ППНД+ИНД+ГПК',(Qgas1-Qwat1)/Qgas1*100)

Qgasall = KPD*gas_streams.at['GTU-PEVD','G']*(gas_streams.at['GTU-PEVD','H']-gas_streams.at['GPK-out','H'])
Qwatall = water_streams.at['PPND-TURB','G']*(water_streams.at['PPND-TURB','H']-water_streams.at['X-GPK','H'])+water_streams.at['PEVD-DROS','G']*(water_streams.at['PEVD-DROS','H']-water_streams.at['X-GPK','H'])-water_streams.at['BND-PEN','G']*(water_streams.at['PEN-EVD','H']-water_streams.at['BND-PEN','H'])
print('dQ/Qsumm',(Qgasall-Qwatall)/Qgasall*100)